In [1]:
import sys
import os

# Check if the notebook is running in Google Colab
def in_colab():
    return 'google.colab' in sys.modules

# If in Colab, create the directory structure and download the required files
if in_colab():
    os.makedirs('graph_gen', exist_ok=True)
    !wget -q https://raw.githubusercontent.com/jojohannsen/langgraph_gen/main/graph_gen/gen_graph.py -O graph_gen/gen_graph.py
    !wget -q https://raw.githubusercontent.com/jojohannsen/langgraph_gen/main/graph_gen/__init__.py -O graph_gen/__init__.py

In [2]:
from graph_gen.gen_graph import gen_graph

# How to wait for user input

One of the main human-in-the-loop interaction patterns is waiting for human input. A key use case involves asking the user clarifying questions. One way to accomplish this is simply go to the END node and exit the graph. Then, any user response comes back in as fresh invocation of the graph. This is basically just creating a chatbot architecture.

The issue with this is it is tough to resume back in a particular point in the graph. Often times the agent is halfway through some process, and just needs a bit of a user input. Although it is possible to design your graph in such a way where you have a `conditional_entry_point` to route user messages back to the right place, that is not super scalable (as it essentially involves having a routing function that can end up almost anywhere).

A separate way to do this is to have a node explicitly for getting user input. This is easy to implement in a notebook setting - you just put an `input()` call in the node. But that isn't exactly production ready.

Luckily, LangGraph makes it possible to do similar things in a production way. The basic idea is:

- Set up a node that represents human input. This can have specific incoming/outgoing edges (as you desire). There shouldn't actually be any logic inside this node.
- Add a breakpoint before the node. This will stop the graph before this node executes (which is good, because there's no real logic in it anyways)
- Use `.update_state` to update the state of the graph. Pass in whatever human response you get. The key here is to use the `as_node` parameter to apply this update **as if you were that node**. This will have the effect of making it so that when you resume execution next it resumes as if that node just acted, and not from the beginning.

**Note:** this requires passing in a checkpointer.

Below is a quick example.

## Setup

First we need to install the packages required

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_anthropic

Next, we need to set API keys for Anthropic (the LLM we will use)

In [3]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")

Optionally, we can set API key for [LangSmith tracing](https://smith.langchain.com/), which will give us best-in-class observability.

In [4]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
_set_env("LANGCHAIN_API_KEY")

## Build the agent

We can now build the agent. We will build a relatively simple ReAct-style agent that does tool calling. We will use Anthropic's models and a fake tool (just for demo purposes).

In [7]:
# Set up the state
from langgraph.graph import MessagesState, START

# Set up the tool
# We will have one real tool - a search tool
# We'll also have one "fake" tool - a "ask_human" tool
# Here we define any ACTUAL tools
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode


@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder for the actual implementation
    # Don't let the LLM know this though 😊
    return [
        f"I looked up: {query}. Result: It's sunny in San Francisco, but you better look out if you're a Gemini 😈."
    ]

tools = [search]
tool_node = ToolNode(tools)

# Set up the model
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-5-sonnet-20240620")

# We are going "bind" all tools to the model
# We have the ACTUAL tools from above, but we also need a mock tool to ask a human
# Since `bind_tools` takes in tools but also just tool definitions,
# We can define a tool definition for `ask_human`
from langchain_core.pydantic_v1 import BaseModel

class AskHuman(BaseModel):
    """Ask the human a question"""
    question: str

model = model.bind_tools(tools + [AskHuman])

# Define nodes and conditional edges
from langchain_core.messages import ToolMessage
from langgraph.prebuilt import ToolInvocation

# Define the function that calls the model
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Build the graph
from langgraph.graph import END, StateGraph

# don't really need this when human is node
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [11]:
def no_tools(state):
    return not state["messages"][-1].tool_calls

def human_needed(state):
    return state["messages"][-1].tool_calls[0]["name"] == "AskHuman"

# the human node
def get_human_input(state):
    last_message = state["messages"][-1]
    tool_call_id = last_message.tool_calls[0]["id"]
    tool_message = last_message.tool_calls[0]
    question = tool_message['args']['question']
    weather_place = input(question)
    tool_message = [
        {"tool_call_id": tool_call_id, "type": "tool", "content": weather_place}
    ]
    return { "messages": tool_message }

graph_spec = """

call_model(MessagesState)
   no_tools => END
   human_needed => get_human_input
   => tool_node

tool_node
   => call_model
   
get_human_input
   => call_model
   
"""

graph_code = gen_graph("wait_user_input", graph_spec, 'memory')
print(graph_code)
exec(graph_code)

wait_user_input = StateGraph(MessagesState)
wait_user_input.add_node('call_model', call_model)
wait_user_input.add_node('tool_node', tool_node)
wait_user_input.add_node('get_human_input', get_human_input)

wait_user_input.set_entry_point('call_model')

def after_call_model(state: MessagesState):
    if no_tools(state):
        return 'END'
    elif human_needed(state):
        return 'get_human_input'
    return 'tool_node'

call_model_dict = {'END': END, 'get_human_input': 'get_human_input', 'tool_node': 'tool_node'}
wait_user_input.add_conditional_edges('call_model', after_call_model, call_model_dict)


wait_user_input.add_edge('tool_node', 'call_model')


wait_user_input.add_edge('get_human_input', 'call_model')




wait_user_input = wait_user_input.compile(checkpointer=memory)


In [12]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(
    content="Use the search tool to ask the user where they are, then look up the weather there"
)
for event in wait_user_input.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Use the search tool to ask the user where they are, then look up the weather there
================================== Ai Message ==================================

[{'text': "Certainly! I'll use the AskHuman tool to ask the user where they are, and then I'll use the search tool to look up the weather in that location. Let's start by asking the user about their location.", 'type': 'text'}, {'id': 'toolu_01K1ojwrEMeTRMcz59JswSZJ', 'input': {'question': 'Where are you currently located?'}, 'name': 'AskHuman', 'type': 'tool_use'}]
Tool Calls:
  AskHuman (toolu_01K1ojwrEMeTRMcz59JswSZJ)
 Call ID: toolu_01K1ojwrEMeTRMcz59JswSZJ
  Args:
    question: Where are you currently located?


Where are you currently located? everywhere


================================= Tool Message =================================

everywhere
================================== Ai Message ==================================

[{'text': 'I see that the user responded with "everywhere." Since this is not a specific location, we\'ll need to ask for a more precise answer to be able to look up the weather. Let\'s ask for a specific city or location.', 'type': 'text'}, {'id': 'toolu_01GwNW7sjAGjTJP4T34sWcrz', 'input': {'question': 'I understand you said "everywhere," but to look up the weather, I need a specific location. Could you please provide a city or town name?'}, 'name': 'AskHuman', 'type': 'tool_use'}]
Tool Calls:
  AskHuman (toolu_01GwNW7sjAGjTJP4T34sWcrz)
 Call ID: toolu_01GwNW7sjAGjTJP4T34sWcrz
  Args:
    question: I understand you said "everywhere," but to look up the weather, I need a specific location. Could you please provide a city or town name?


I understand you said "everywhere," but to look up the weather, I need a specific location. Could you please provide a city or town name? yuma


================================= Tool Message =================================

yuma
================================== Ai Message ==================================

[{'text': "Thank you for providing a specific location. Now that we know the user is in Yuma, let's use the search tool to look up the weather there.", 'type': 'text'}, {'id': 'toolu_012vWV12PNbWHGC829jBka6z', 'input': {'query': 'current weather in Yuma'}, 'name': 'search', 'type': 'tool_use'}]
Tool Calls:
  search (toolu_012vWV12PNbWHGC829jBka6z)
 Call ID: toolu_012vWV12PNbWHGC829jBka6z
  Args:
    query: current weather in Yuma
================================= Tool Message =================================
Name: search

["I looked up: current weather in Yuma. Result: It's sunny in San Francisco, but you better look out if you're a Gemini \ud83d\ude08."]
================================== Ai Message ==================================

[{'text': "I apologize, but it seems that the search results didn't provide accurate

I'm sorry, but I'm having trouble getting accurate weather information for Yuma through the search tool. The results are not relevant to your location. Would you like me to try searching for general information about Yuma instead, or do you have another question I can help you with? no


================================= Tool Message =================================

no
================================== Ai Message ==================================

I understand. Since you've answered "no" to my question about searching for general information about Yuma or helping with another question, I'll wrap up our conversation here. 

To summarize:
1. We determined that you're located in Yuma.
2. We attempted to search for weather information in Yuma, but encountered issues with getting accurate results.
3. You declined further assistance or alternative searches.

Is there anything else you'd like to know or discuss? If not, feel free to ask if you have any other questions in the future. I'm here to help!
